In [88]:
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.convolution import convolve_fft, Gaussian2DKernel
import os
import glob
from glob import glob
from astropy.table import Table
mass_types = [(-1.,  -1., 'dm', 'black'),
              (-1.,  -1.,    'stars', 'goldenrod'),
              (0.,    1.5e4, 'cold', 'darkblue'),
              (1.5e4, 1.e5, 'warm', 'blue'),
              (1.e5,  1.e6, 'warmhot', 'red'),
              (1.e6,  1.e10, 'hot', 'darkred')]
mass_types = ['dm', 'stars', 'cold', 'warm', 'warmhot', 'hot']

halos = ['8508',
         '5016',
         '4123',
         '2392',
         '5036',
         '2878']

cmp = plt.cm.viridis
cmp.set_bad('k')


<ipython-input-88-eea44c7f4d77>:24: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("viridis"))
  cmp.set_bad('k')


In [187]:
matplotlib.rcParams.update({'font.size': 12})
for halo in halos[:1]:
    DD_fls = sort(glob('/Users/rsimons/Dropbox/foggie/angular_momentum/new_profiles_temp/%s/Lprof_%s_DD????.npy'%(halo, halo)))
    #DD_fls = sort(glob('/Users/rsimons/Dropbox/foggie/angular_momentum/profiles/%s/Lprof_%s_DD????.npy'%(halo, halo)))
    rvir_masses = Table.read('/Users/rsimons/Dropbox/git/foggie/foggie/halo_infos/00%s/nref11c_nref9f/rvir_masses.hdf5'%halo)
    for DD_fl in DD_fls:
        DD = DD_fl.split('/')[-1][-10:-4]
        #DD = DD_fl.split('/')[-1][-15:-9]
        print (DD)
        #rvir_test = rvir_masses['radius'][rvir_masses['snapshot'] == DD].data
        #redshift  = rvir_masses['redshift'][rvir_masses['snapshot'] == DD].data
        #print (halo, DD, '%.2f'%redshift) 
        fig, axes = plt.subplots(1,2, figsize = (8, 4), facecolor = 'white')
        ax1 = axes[0]
        ax2 = axes[1]
        figname = DD_fl.replace('.npy', '.png').replace('profiles', 'figures')
        Lprof = np.load(DD_fl, allow_pickle = True)[()]

        for mm, mtype in enumerate(mass_types[2:3]):
            #if mtype != 'stars': continue
            
            Lhst_full = Lprof[mtype]['r_dist']['L_hst'][:3,:,:,:]
            Mhst_full = Lprof[mtype]['r_dist']['M_hst'][:3,:,:,:]
            dim_tuple = tuple(np.arange(Lhst_full.ndim-2))
            Lhst_center  = np.rot90(np.nansum(Lhst_full, axis = dim_tuple))
            Mhst  = np.rot90(np.nansum(Mhst_full, axis = dim_tuple))     
            
            Lhst_rvl = Lhst_center.ravel()
            vmn, vmx = 0.0, np.nanpercentile(Lhst_rvl, [99.5])[0]
            axes[0].imshow(Lhst_center, cmap = cmp, vmin = vmn, vmax = vmx)
            axes[0].annotate(mtype, (0.98, 0.05), xycoords = 'axes fraction', ha = 'right', va = 'bottom', \
                        color = 'white', fontweight = 'bold', fontsize = 22)
            
            Lhst_full = Lprof['warm']['r_dist']['L_hst'][3:10,:2,:1,:]
            Mhst_full = Lprof['warm']['r_dist']['M_hst'][3:10,:2,:1,:]
            dim_tuple = tuple(np.arange(Lhst_full.ndim-2))
            Lhst  = np.rot90(np.nansum(Lhst_full, axis = dim_tuple))
            Mhst  = np.rot90(np.log10(np.nansum(Mhst_full, axis = dim_tuple)))
            
            Lhst_rvl = Lhst.ravel()
            Mhst_rvl = Mhst.ravel()
            if len(Mhst_rvl[isfinite(Mhst_rvl)]) < 3: continue
            vmn, vmx = np.nanpercentile(Mhst_rvl[isfinite(Mhst_rvl)], [5])[0], np.nanpercentile(Mhst_rvl[isfinite(Mhst_rvl)], [95])[0]

            axes[1].imshow(Mhst, cmap = cmp, vmin = vmn, vmax = vmx)
            axes[1].annotate('warm', (0.98, 0.05), xycoords = 'axes fraction', ha = 'right', va = 'bottom', \
                        color = 'white', fontweight = 'bold', fontsize = 22)
                        
            
            
            
        for ax in axes:
            ax.set_xlim(0, 199)
            ax.set_ylim(199, 0)

            ax.set_xlabel(r'$\theta_{\mathrm{L}}$ (deg.)')
            ax.set_ylabel(r'$\phi_{\mathrm{L}}$ (deg.)')

            xtck_use  = np.arange(-180, 240, 60)
            xtcks_real = np.interp(xtck_use, [-180, 180],  [0, 199])
            ax.set_xticks(xtcks_real)
            ax.set_xticklabels(['%i'%tck for tck in xtck_use])

            ytck_use  = np.arange(0, 240, 60)
            ytcks_real = np.interp(ytck_use, [0, 180],  [0, 199])
            ax.set_yticks(ytcks_real)
            ax.set_yticklabels(['%i'%tck for tck in ytck_use])

            binsx = np.arange(199)#(xmn, xmx, nbins)
            binsy = np.arange(199)#(ymn, ymx, nbins)

            YY, XX = np.meshgrid(binsy, binsx)

        axes[1].contour(YY, XX, Lhst_center/np.max(Lhst_center), 
                   levels = [0.10], zorder = 10, colors = 'white', 
                   alpha = 0.5, linewidth = 0.5)

        
        fig.subplots_adjust(hspace = 0.05, wspace = 0.05)
        fig.set_dpi(300)
        fig.tight_layout()
        fig.savefig(figname)
        plt.close(fig)




DD0487


<ipython-input-187-320f21ea2a8c>:38: RuntimeWarning: divide by zero encountered in log10
  Mhst  = np.rot90(np.log10(np.nansum(Mhst_full, axis = dim_tuple)))
<ipython-input-187-320f21ea2a8c>:74: UserWarning: The following kwargs were not used by contour: 'linewidth'
  axes[1].contour(YY, XX, Lhst_center/np.max(Lhst_center),


DD0537
DD0587
DD0637
DD0687
DD0737
DD0787
DD0837
DD0887
DD0937
DD0987
DD1037
DD1087
DD1137
DD1187
DD1237
DD1287
DD1337
DD1387
DD1437
DD1487
DD1537
DD1587
DD1637
DD1687
DD1737
DD1787
DD1837
DD1887
DD1937
DD1987
DD2037
DD2087
DD2137
DD2187
DD2237
DD2287
DD2337
DD2387
DD2427


In [184]:
Mhst_rvl[isfinite(Mhst_rvl)]

array([], dtype=float64)

In [170]:
Mhst_rvl[isfinite(Mhst_rvl)]

array([3.42800295, 3.39263728, 3.17152067, ..., 3.1054272 , 2.91663339,
       2.6384473 ])

In [ ]:
axes[1].contour

In [89]:
for halo in halos[:1]:
    DD_fls = sort(glob('/Users/rsimons/Dropbox/foggie/angular_momentum/new_profiles_temp/%s/Lprof_%s_DD????.npy'%(halo, halo)))
    #DD_fls = sort(glob('/Users/rsimons/Dropbox/foggie/angular_momentum/profiles/%s/Lprof_%s_DD????_test.npy'%(halo, halo)))
    rvir_masses = Table.read('/Users/rsimons/Dropbox/git/foggie/foggie/halo_infos/00%s/nref11c_nref9f/rvir_masses.hdf5'%halo)
    for DD_fl in DD_fls:
        DD = DD_fl.split('/')[-1][-10:-4]
        #DD = DD_fl.split('/')[-1][-15:-9]
        print (DD)
        #rvir_test = rvir_masses['radius'][rvir_masses['snapshot'] == DD].data
        #redshift  = rvir_masses['redshift'][rvir_masses['snapshot'] == DD].data
        #print (halo, DD, '%.2f'%redshift) 
        fig, axes = plt.subplots(2,3, figsize = (9, 6), facecolor = 'white')
        figname = DD_fl.replace('.npy', '.png').replace('profiles', 'figures')
        Lprof = np.load(DD_fl, allow_pickle = True)[()]

        for mm, mtype in enumerate(mass_types):
            #if mtype != 'stars': continue
            ax = axes.ravel()[mm]
            Lhst_full = Lprof[mtype]['r_dist']['L_hst'][:2,:,:,:]
            Mhst_full = Lprof[mtype]['r_dist']['M_hst'][:2,:,:,:]
            dim_tuple = tuple(np.arange(Lhst_full.ndim-2))
            Lhst  = np.rot90(np.nansum(Lhst_full, axis = dim_tuple))
            Mhst  = np.rot90(np.nansum(Mhst_full, axis = dim_tuple))     
            
            Lhst_rvl = Lhst.ravel()
            vmn, vmx = 0.0, np.nanpercentile(Lhst_rvl, [99.5])[0]
            ax.imshow(Lhst, cmap = cmp, vmin = vmn, vmax = vmx)
            ax.annotate(mtype, (0.98, 0.05), xycoords = 'axes fraction', ha = 'right', va = 'bottom', \
                        color = 'white', fontweight = 'bold', fontsize = 22)

        fig.subplots_adjust(hspace = 0.05, wspace = 0.05)
        fig.set_dpi(300)
        fig.tight_layout()
        fig.savefig(figname)
        plt.close(fig)




DD0487
DD0537
DD0587
DD0637
DD0687
DD0737
DD0787
DD0837
DD0887
DD0937
DD0987
DD1037
DD1087
DD1137
DD1187
DD1237
DD1287
DD1337
DD1387
DD1437
DD1487
DD1537
DD1587
DD1637
DD1687
DD1737
DD1787
DD1837
DD1887
DD1937
DD1987
DD2037
DD2087
DD2137
DD2187
DD2237
DD2287
DD2337
DD2387
DD2427


In [75]:
Lx = Lprof['stars']['rprof']['Lx'][1]
Ly = Lprof['stars']['rprof']['Ly'][1]
Lz = Lprof['stars']['rprof']['Lz'][1]
print (Lx, Ly, Lz)
Ltot = np.sqrt(Lx**2. + Ly**2. + Lz**2.)
thel = np.arctan2(Ly,Lx)*180./pi
phil = np.arccos(Lz/Ltot)*180./pi
print (thel, phil)

-7.064078152193382e+71 cm**2*g/s -8.287593643451987e+71 cm**2*g/s -9.078706194499315e+71 cm**2*g/s
-130.44320586101756 dimensionless 129.8178721044258


In [ ]:
fig2, axes2 = plt.subplots(1,1, figsize = (5, 5), facecolor = 'white')
figname2 = '/Users/rsimons/Dropbox/foggie/angular_momentum/figures/8508/combined.png'
for DD in [500, 600, 700, 800, 900, 1000, 1100, 1346, 1600, 2427][-2:-1]:
#for (clr_contour, DD) in [('darkred', 500), ('darkblue', 600), ('blue', 800), ('lightblue', 1346), ('black', 2427)]:
#for (clr_contour, DD) in [('lightblue', 1346)]:
    fl = '/Users/rsimons/Dropbox/foggie/angular_momentum/profiles/8508/Lprof_8508_DD%.4i.npy'%DD

    L_all = np.load(fl, allow_pickle = True)[()]

    for rr, rmin in enumerate(np.arange(0, 100, 0.5)):
        rmax = rmin + 1.
        fig, axes = plt.subplots(2,3, figsize = (9, 6), facecolor = 'white')
        figname = fl.replace('npy', '%.3i.png'%(rr)).replace('profiles', 'figures')

        for nn, (low_temp, high_temp, name, clr) in enumerate(mass_types):
            ax = axes.ravel()[nn]
            for (ix, jx, kx) in [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0)]:
                phil = np.arccos(kx)*180./pi
                thel = np.arctan2(jx, ix)*180./pi
                ax.plot(thel, phil, 'x', color = 'grey')
            ax.plot()
            nbins = 150
            ax.axhspan(xmin = 0, xmax = 1, ymin = 175, ymax = 180, zorder = 12, color = 'grey', alpha = 0.5)
            ax.axhspan(xmin = 0, xmax = 1, ymin = 0, ymax = 5,     zorder = 12, color = 'grey', alpha = 0.5)

            xvar, xmn, xmx, xlbl =           L_all[name]['adist']['thel'], -180, 180, r'$\theta_{\mathrm{L}}$ (deg.)'
            yvar, ymn, ymx, ylbl =           L_all[name]['adist']['phil'], 0, 180, r'$\phi_{\mathrm{L}}$ (deg.)'
            weights = L_all[name]['adist']['ltot']
            r = L_all[name]['adist']['r']
            gd      =   (r > rmin) & (r < rmax)
            xvar    =    xvar[gd]
            yvar    =    yvar[gd]
            weights =    weights[gd]
            
            cmap = plt.cm.viridis
            binsx = np.linspace(xmn, xmx, nbins)
            binsy = np.linspace(ymn, ymx, nbins)
            hst = histogram2d(xvar, yvar, nbins, [[xmn, xmx], [ymn, ymx]], weights = weights)[0]
            YY, XX = np.meshgrid(binsy, binsx)
            
            vmin = 0.0
            hst_rvl = hst.ravel()
            if len(hst_rvl[hst_rvl > 0.0]) > 5:
                vmax = np.nanpercentile(hst_rvl[hst_rvl > 0.0], [98])[0]
                
            else: vmax = 1.0
            x = ax.hist2d(xvar, yvar, bins = [binsx, binsy], \
                      norm = matplotlib.colors.Normalize(vmin, vmax),\
                      weights = weights, cmap = cmap)
            if (nn == 1) & (rr == 0):
                kern = Gaussian2DKernel(1)
                hst_center = convolve_fft(hst, kern)
                #axes2.contour(XX, YY, hst_center, levels = 3, zorder = 10, colors = clr_contour, origin = 'upper')
                

                
            #if (nn == 2): 
            #    axes2.contourf(XX, YY, hst, levels = 1, colors = clr_contour, 
            #                  zorder = 0, linestyles = 'dashed', origin = 'upper')
 
            ax.annotate(name, (0.98, 0.05), xycoords = 'axes fraction', ha = 'right', va = 'bottom', \
                        color = 'white', fontweight = 'bold', fontsize = 22)
        axes[0,2].annotate('%.1f $<$ r (kpc) $<$ %.1f'%(rmin, rmax), (0.98, 0.98), xycoords = 'axes fraction', ha = 'right', va = 'top', \
                    color = 'white', fontweight = 'bold', fontsize = 15)

        for ax in axes[:,0]:
            ax.set_ylabel(ylbl, fontsize = 15)
        for ax in axes[:,1:].ravel():
            ax.set_yticklabels([''])
       
        for ax in axes.ravel(): ax.contour(XX, YY, hst_center/np.max(hst_center), levels = [0.10, 0.40], zorder = 10, colors = 'white', alpha = 0.4, linewidth = 1, origin = 'upper')

        for ax in axes[1]:ax.set_xlabel(xlbl, fontsize = 15)
        for ax in axes[0].ravel():
            ax.set_xticklabels([''])
            ax.set_yticks([0, 45, 90, 135, 180])

        axes2.set_ylabel(ylbl, fontsize = 15)            
        axes2.set_xlabel(xlbl, fontsize = 15)
        axes2.set_yticks([0, 45, 90, 135, 180])
        fig.subplots_adjust(hspace = 0.05, wspace = 0.05)
        fig.set_dpi(300)
        fig.tight_layout()
        fig.savefig(figname)
        plt.close(fig)
    DDname = 'DD'+ '%.4i'%DD
    print('ffmpeg -r 12 -f image2 -s 2700x1800 -start_number 0 -i Lprof_8508_' + DDname + r'.%03d.png -vframes 1000 -vcodec libx264  -pix_fmt yuv420p Lprof_8508_' + DDname + '.mp4')
fig2.set_dpi(300)
fig2.savefig(figname2)
plt.close(fig2)


<ipython-input-34-791e29810f27>:68: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''])
<ipython-input-34-791e29810f27>:70: UserWarning: The following kwargs were not used by contour: 'linewidth'
  for ax in axes.ravel(): ax.contour(XX, YY, hst_center/np.max(hst_center), levels = [0.10, 0.40], zorder = 10, colors = 'white', alpha = 0.4, linewidth = 1, origin = 'upper')
<ipython-input-34-791e29810f27>:74: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''])


In [80]:
ds.derived_field_list

NameError: name 'ds' is not defined

In [85]:
Lprof.keys()

dict_keys(['props', 'cold', 'warm', 'warmhot', 'hot', 'stars', 'dm'])

In [143]:
ax.contour?

In [67]:
hst = np.histogramdd(xvar, yvar, bins = [binsx, binsy],weights = weights) 

TypeError: _histogramdd_dispatcher() got multiple values for argument 'bins'

In [107]:
plt.cm.Blues?

In [ ]:
axes2[0].contour